best tuned settings:

    *  R2 for test grs data: 0.8187043576395236

    *  negMedAE for test grs data: -5419.092768984354

    *  R2 for test gcs data: 0.7201367512319357

    *  negMedAE for test gcs data: -7870.347244399914

In [2]:
from helpers import utils
from os.path import join, dirname
from dotenv import load_dotenv
import os
import pickle
from snowflake import connector
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.ticker import FormatStrFormatter, StrMethodFormatter
from scipy.optimize import curve_fit
from sklearn.preprocessing import Normalizer, QuantileTransformer, RobustScaler
from sklearn.pipeline import Pipeline
from sklearn.model_selection import GridSearchCV, train_test_split, RepeatedKFold
from sklearn.ensemble import RandomForestRegressor
from sklearn.feature_selection import SelectKBest, f_regression
from sklearn.ensemble import AdaBoostRegressor
from sklearn.linear_model import ElasticNetCV
from tpot.builtins import StackingEstimator
from sklearn.metrics import r2_score, make_scorer


pd.options.display.float_format = '{:,.2f}'.format

# get environment variables
dotenv_path = join(dirname('streamlit_grs_fit\\app\\'), '.env')
load_dotenv(dotenv_path)
SF_ACCOUNT = os.getenv('SF_ACCOUNT')
SF_USER = os.getenv('SF_USER')
SF_PASSWORD = os.getenv('SF_PASSWORD')
SF_ROLE = os.getenv('SF_ROLE')
SF_WAREHOUSE = os.getenv('SF_WAREHOUSE')
SF_DATABASE = os.getenv('SF_DATABASE')
SF_SCHEMA = os.getenv('SF_SCHEMA')

def load_data(query):
    conn = connector.connect(
        user = SF_USER
        ,password = SF_PASSWORD
        ,account = SF_ACCOUNT
        ,warehouse = SF_WAREHOUSE
        ,database = SF_DATABASE
        ,schema = SF_SCHEMA
        ,role = SF_ROLE
    )
    cur = conn.cursor()
    df_data = cur.execute(query).fetch_pandas_all()
    return df_data

c:\Users\SwanS\Anaconda3\envs\grs_fit\lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [6]:
query = 'select '+\
            'JOB'+\
            ',DIRECT_COST'+\
            ',DIV_00_DIRECT_COST'+\
            ',DIV_01_DIRECT_COST'+\
            ',DIV_02_DIRECT_COST'+\
            ',DIV_03_DIRECT_COST'+\
            ',DIV_04_DIRECT_COST'+\
            ',DIV_05_DIRECT_COST'+\
            ',DIV_06_DIRECT_COST'+\
            ',DIV_07_DIRECT_COST'+\
            ',DIV_08_DIRECT_COST'+\
            ',DIV_09_DIRECT_COST'+\
            ',DIV_10_DIRECT_COST'+\
            ',DIV_11_DIRECT_COST'+\
            ',DIV_12_DIRECT_COST'+\
            ',DIV_13_DIRECT_COST'+\
            ',DIV_14_DIRECT_COST'+\
            ',DIV_15_DIRECT_COST'+\
            ',DIV_16_DIRECT_COST'+\
            ',DIV_17_DIRECT_COST'+\
            ',DIV_18_DIRECT_COST'+\
            ',DIV_19_DIRECT_COST'+\
            ',DIV_21_DIRECT_COST'+\
            ',DIV_22_DIRECT_COST'+\
            ',DIV_23_DIRECT_COST'+\
            ',DIV_26_DIRECT_COST'+\
            ',DIV_27_DIRECT_COST'+\
            ',DIV_28_DIRECT_COST'+\
            ',DIV_31_DIRECT_COST'+\
            ',DIV_32_DIRECT_COST'+\
            ',DIV_33_DIRECT_COST'+\
            ',DIV_34_DIRECT_COST'+\
            ',DIV_55_DIRECT_COST'+\
            ',GCS_COST'+\
            ',GRS_COST '+\
            'from sandbox.global.ml_grs_fit ' 
df_data = load_data(query).set_index('JOB') 
df_data = pd.DataFrame(df_data)
df_data = df_data.fillna(0)

In [7]:
df_working = df_data.loc[
                    (0 != df_data.GRS_COST) &
                    (0 != df_data.GCS_COST)
].copy()
df_working.describe()

,DIRECT_COST,DIV_00_DIRECT_COST,DIV_01_DIRECT_COST,DIV_02_DIRECT_COST,DIV_03_DIRECT_COST,DIV_04_DIRECT_COST,DIV_05_DIRECT_COST,DIV_06_DIRECT_COST,DIV_07_DIRECT_COST,DIV_08_DIRECT_COST,...,DIV_26_DIRECT_COST,DIV_27_DIRECT_COST,DIV_28_DIRECT_COST,DIV_31_DIRECT_COST,DIV_32_DIRECT_COST,DIV_33_DIRECT_COST,DIV_34_DIRECT_COST,DIV_55_DIRECT_COST,GCS_COST,GRS_COST
count,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00",...,"3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00","3,332.00"
mean,"2,753,305.18","15,416.14","1,095.30","120,305.74","305,087.19","25,975.15","192,784.27","83,875.74","97,562.86","262,156.35",...,"279,623.43","22,192.54","7,159.77","84,671.51","18,546.22","19,750.99",505.83,"1,710.05","159,975.26","130,762.61"
std,"18,205,353.59","247,803.94","21,856.71","835,518.53","2,456,075.21","255,679.11","1,926,522.10","479,208.15","712,596.43","2,347,850.24",...,"2,283,423.57","355,302.65","98,050.42","852,341.42","200,217.24","315,920.99","14,397.44","98,425.42","991,159.86","893,014.97"
min,"-3,404,153.00","-327,264.32",0.00,"-41,282.56","-4,315,458.20","-164,664.12","-1,398,362.17","-174,550.72","-16,197.24","-407,656.41",...,-104.77,0.00,0.00,"-2,322.10",-11.50,"-53,330.53",-4.20,0.00,"-3,335,087.00","-629,785.00"
25%,"4,410.75",0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,554.25,305.75
50%,"31,937.00",0.00,0.00,889.22,0.00,0.00,0.00,353.55,0.00,37.68,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,"4,763.00","2,099.00"
75%,"279,720.25",0.00,0.00,"10,486.36","1,041.34",0.00,180.50,"10,992.53",44.95,"9,230.26",...,"3,893.87",0.00,0.00,0.00,0.00,0.00,0.00,0.00,"35,369.50","16,783.00"
max,"399,570,438.00","8,618,727.16","762,087.69","26,715,320.64","49,560,386.64","9,635,075.36","51,850,542.49","9,711,287.61","19,337,427.69","56,887,683.67",...,"57,604,860.12","14,904,687.98","2,706,150.04","19,786,153.90","6,898,015.60","10,785,799.69","607,500.00","5,681,453.04","23,749,853.00","18,182,714.00"


In [13]:
X  = df_working.iloc[:,:-2] #.values
y_gcs = df_working.iloc[:,-2:-1].values.ravel()
y_grs = df_working.iloc[:,-1:].values.ravel()
y_grs.shape

(3332,)

In [14]:
X_train, X_test, y_grs_train, y_grs_test = train_test_split(X, y_grs, test_size=0.33, random_state=42)
X_train, X_test, y_gcs_train, y_gcs_test = train_test_split(X, y_gcs, test_size=0.33, random_state=42)

In [15]:
from sklearn.preprocessing import MinMaxScaler


cv = RepeatedKFold(n_splits=10, n_repeats=3, random_state=42)

# pipeline setup
pipeline = Pipeline([
                     ('scaler', None) 
                     ,('kbest', SelectKBest(f_regression))
                     ,('stack_reg_one', StackingEstimator(estimator=AdaBoostRegressor()))
                    #  ,('stack_reg_two', StackingEstimator(estimator=ElasticNetCV()))  #this makes it worse
                     ,('regressor', RandomForestRegressor())
                     ])

parameters = {
                'scaler':  [RobustScaler(), MinMaxScaler()]
                ,'kbest__k': [6]
                ,'stack_reg_one__estimator__learning_rate': [0.04, 0.1]
                ,'stack_reg_one__estimator__loss': ['linear', 'exponential']
                ,'stack_reg_one__estimator__n_estimators': [100, 150] 
                # ,'stack_reg_two__estimator__l1_ratio': [0.1] 
                # ,'stack_reg_two__estimator__tol': [0.001] 
                ,'regressor__bootstrap': [True, False]
                ,'regressor__max_features': [0.1]
                ,'regressor__min_samples_leaf': [1]
                ,'regressor__min_samples_split': [7]
                ,'regressor__n_estimators': [100]
                }
#grs model
grs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=cv
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(utils.neg_median_absolute_error)
    }
    ,refit=utils.refit_strategy
    ,return_train_score=False
    ,n_jobs=-2
)   

#gcs model
gcs_grid = GridSearchCV(
    pipeline
    ,parameters
    ,cv=cv
    ,scoring={'R2': make_scorer(r2_score)
            ,'negMedAE': make_scorer(utils.neg_median_absolute_error)
    }
    ,refit=utils.refit_strategy
    ,return_train_score=False
    ,n_jobs=-2
)   

In [16]:
grs_grid = grs_grid.fit(X_train, y_grs_train)

All grid-search results:
R2: 0.607 (±0.623), negMedAE: -4021.049 (±601.892), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 100, 'scaler': RobustScaler(), 'stack_reg_one__estimator__learning_rate': 0.04, 'stack_reg_one__estimator__loss': 'linear', 'stack_reg_one__estimator__n_estimators': 100}
R2: 0.589 (±0.716), negMedAE: -4015.151 (±542.983), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 100, 'scaler': RobustScaler(), 'stack_reg_one__estimator__learning_rate': 0.04, 'stack_reg_one__estimator__loss': 'linear', 'stack_reg_one__estimator__n_estimators': 150}
R2: 0.591 (±0.733), negMedAE: -4119.967 (±671.074), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regr

In [20]:
gcs_grid = gcs_grid.fit(X_train, y_gcs_train)

All grid-search results:
R2: 0.583 (±0.266), negMedAE: -7262.867 (±1018.852), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 100, 'scaler': RobustScaler(), 'stack_reg_one__estimator__learning_rate': 0.04, 'stack_reg_one__estimator__loss': 'linear', 'stack_reg_one__estimator__n_estimators': 100}
R2: 0.589 (±0.269), negMedAE: -7480.323 (±1175.225), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 100, 'scaler': RobustScaler(), 'stack_reg_one__estimator__learning_rate': 0.04, 'stack_reg_one__estimator__loss': 'linear', 'stack_reg_one__estimator__n_estimators': 150}
R2: 0.604 (±0.263), negMedAE: -7420.455 (±1275.330), for {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'r

In [18]:
y_grs_test_pred = grs_grid.best_estimator_.predict(X_test)
print(f'R2 for test grs data: {r2_score(y_grs_test, y_grs_test_pred)}')
print(f'negMedAE for test grs data: {utils.neg_median_absolute_error(y_grs_test, y_grs_test_pred)}')

R2 for test grs data: 0.8435848317647123
negMedAE for test grs data: -5362.528629528631


In [21]:
y_gcs_test_pred = gcs_grid.best_estimator_.predict(X_test)
print(f'R2 for test gcs data: {r2_score(y_gcs_test, y_gcs_test_pred)}')
print(f'negMedAE for test gcs data: {utils.neg_median_absolute_error(y_gcs_test, y_gcs_test_pred)}')

R2 for test gcs data: 0.7136713606030922
negMedAE for test gcs data: -8257.74693466455


In [22]:
print("the best grs estimator is \n {} ".format(grs_grid.best_estimator_))
print("the best grs parameters are \n {}".format(grs_grid.best_params_))
print("the best gcs estimator is \n {} ".format(gcs_grid.best_estimator_))
print("the best gcs parameters are \n {}".format(gcs_grid.best_params_))

the best grs estimator is 
 Pipeline(steps=[('scaler', MinMaxScaler()),
                ('kbest',
                 SelectKBest(k=6,
                             score_func=<function f_regression at 0x000001AA21EDDD30>)),
                ('stack_reg_one',
                 StackingEstimator(estimator=AdaBoostRegressor(learning_rate=0.04,
                                                               loss='exponential',
                                                               n_estimators=100))),
                ('regressor',
                 RandomForestRegressor(max_features=0.1, min_samples_split=7))]) 
the best grs parameters are 
 {'kbest__k': 6, 'regressor__bootstrap': True, 'regressor__max_features': 0.1, 'regressor__min_samples_leaf': 1, 'regressor__min_samples_split': 7, 'regressor__n_estimators': 100, 'scaler': MinMaxScaler(), 'stack_reg_one__estimator__learning_rate': 0.04, 'stack_reg_one__estimator__loss': 'exponential', 'stack_reg_one__estimator__n_estimators': 100}
the

In [23]:
grs_best_pipe = grs_grid.best_estimator_
grs_mask = list(grs_best_pipe.fit(X,y_grs)['kbest'].get_feature_names_out())
grs_mask = [int(_.replace('x', '')) for _ in grs_mask]
grs_model = grs_best_pipe.fit(df_working.iloc[:,grs_mask],y_grs)
grs_predictions = grs_model.predict(df_working.iloc[:,grs_mask])

In [67]:
df_working.iloc[:,[int(_.replace('x', '')) for _ in grs_mask]]

,DIRECT_COST,DIV_03_DIRECT_COST,DIV_05_DIRECT_COST,DIV_07_DIRECT_COST,DIV_08_DIRECT_COST,DIV_31_DIRECT_COST
JOB,,,,,,
5344,516.00,0.00,0.00,0.00,0.00,0.00
7901,"2,047,211.00",0.00,"14,276.46","9,603.00","122,926.75",0.00
4328,"11,221,450.00","546,160.67","619,551.67","369,135.84","605,646.26",0.00
7882,"39,588.00","22,941.99",0.00,0.00,0.00,0.00
4294.053,"3,894.00",0.00,0.00,0.00,0.00,0.00
...,...,...,...,...,...,...
4673,"1,367.00",0.00,0.00,0.00,0.00,0.00
S4955,"8,167.00",0.00,15.48,0.00,0.00,0.00
6201,"1,316.00",0.00,0.00,0.00,0.00,0.00


In [24]:
gcs_best_pipe = gcs_grid.best_estimator_
gcs_mask = list(gcs_best_pipe.fit(X,y_gcs)['kbest'].get_feature_names_out())
gcs_mask = [int(_.replace('x', '')) for _ in gcs_mask]
gcs_model = gcs_best_pipe.fit(df_working.iloc[:,gcs_mask],y_gcs)
gcs_predictions = gcs_model.predict(df_working.iloc[:,gcs_mask])

In [25]:
grs_parameters = list(df_working.iloc[:,grs_mask].columns)
gcs_parameters = list(df_working.iloc[:,gcs_mask].columns)
combined_mask = list(set(grs_parameters + gcs_parameters))
df = df_working[combined_mask].copy()
df['GRS_TRUE'] = df_working.iloc[:,-1:]
df['GCS_TRUE'] = df_working.iloc[:,-2:-1]
df['GRS_PREDICTIONS'] = grs_predictions
df['GCS_PREDICTIONS'] = gcs_predictions
rfr_model_bag = {
    'df': df
    ,'grs_model': grs_model
    ,'grs_parameters': grs_parameters
    ,'gcs_model': gcs_model
    ,'gcs_parameters': gcs_parameters
}
with open('./app/rfr_model_bag.pkl','wb') as p:
    pickle.dump(rfr_model_bag, p, protocol=-1)

In [26]:
with open('./app/rfr_model_bag.pkl','rb') as p:
    bag = pickle.load(p)

In [27]:
bag.keys()

dict_keys(['df', 'grs_model', 'grs_parameters', 'gcs_model', 'gcs_parameters'])

In [28]:
grs_params = bag['grs_parameters']
gcs_params = bag['gcs_parameters']
all_params = list(set(grs_params + gcs_params))
test_vec = bag['df'][all_params].sample(1).copy()
print(*list(test_vec[grs_params].columns), sep='\n,')

DIRECT_COST
,DIV_03_DIRECT_COST
,DIV_05_DIRECT_COST
,DIV_07_DIRECT_COST
,DIV_08_DIRECT_COST
,DIV_31_DIRECT_COST


In [29]:
bag['df']

,DIRECT_COST,DIV_07_DIRECT_COST,DIV_08_DIRECT_COST,DIV_05_DIRECT_COST,DIV_03_DIRECT_COST,DIV_09_DIRECT_COST,DIV_21_DIRECT_COST,DIV_31_DIRECT_COST,GRS_TRUE,GCS_TRUE,GRS_PREDICTIONS,GCS_PREDICTIONS
JOB,,,,,,,,,,,,
5344,516.00,0.00,0.00,0.00,0.00,264.48,0.00,0.00,532.00,772.00,"1,206.11",520.20
7901,"2,047,211.00","9,603.00","122,926.75","14,276.46",0.00,"54,784.81","6,700.00",0.00,"332,812.00","662,620.00","201,480.83","362,970.20"
4328,"11,221,450.00","369,135.84","605,646.26","619,551.67","546,160.67","1,594,048.88",0.00,0.00,"695,547.00","742,756.00","501,800.82","749,844.14"
7882,"39,588.00",0.00,0.00,0.00,"22,941.99",0.00,0.00,0.00,"9,105.00","12,046.00","6,167.22","8,831.60"
4294.053,"3,894.00",0.00,0.00,0.00,0.00,"2,865.44",0.00,0.00,"2,449.00",976.00,941.64,861.96
...,...,...,...,...,...,...,...,...,...,...,...,...
4673,"1,367.00",0.00,0.00,0.00,0.00,0.00,0.00,0.00,-508.00,"31,098.00","1,384.80","10,087.55"
S4955,"8,167.00",0.00,0.00,15.48,0.00,"6,710.91",0.00,0.00,896.00,270.00,"10,070.07","5,380.53"
6201,"1,316.00",0.00,0.00,0.00,0.00,0.00,"1,316.00",0.00,391.00,88.00,"1,363.37","4,914.11"


In [10]:
gcs_true = df_working.iloc[:,-2:-1]
grs_true = df_working.iloc[:,-1:]
grs_true.describe()

,GRS_COST
count,"3,332.00"
mean,"130,762.61"
std,"893,014.97"
min,"-629,785.00"
25%,305.75
50%,"2,099.00"
75%,"16,783.00"
max,"18,182,714.00"


In [12]:
r2_score(grs_true, bag['df'].GRS_PREDICTIONS)

0.9496829599337434